In [153]:
import gauleg as gl 
import sympy as sp 
import numpy as np 
import pandas as pd 
import math
import matplotlib.pyplot as plt
import Solver as sl 
from scipy.interpolate import interp1d

%load_ext autoreload

%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generating Delta Vector

In [154]:
def interpolation_for_u_h(nodal, mesh, num_points):
    """
    Takes in the solution of the forward solver at high resolution and returns the 
    corresponing values of u_h at a set xi that is predetermined 

    Returns: 
    Array of observation u_h that corresponds to each xi using interpolation if xi not 
    in final mesh. 
    """
    nodal = sl.assemble_nodal_values(nodal)
    mesh = np.array(mesh)  # Replace with your mesh node coordinates
    nodal = np.array(nodal).flatten()  # Replace with your computed u_h values   
    x_min = 0.0
    x_max = 1.0
    xi = np.linspace(x_min, x_max, num_points)

    u_h_interp = interp1d(mesh, nodal, kind='linear', fill_value="extrapolate")
    
    # Evaluate u_h at the observation points
    u_h_at_xi = u_h_interp(xi)

    return u_h_at_xi

def cov_matrix(sigma, num_points):
    sigma = (sigma ** 2) * np.eye(num_points)
    return sigma



In [155]:
def add_noise(observations_at_xi, num_points, sigma):
    """
    Adds a normally distributed noise, theta
    to observations from the forward solver.

    Arguments:
    observations_at_xi : observations at predetermined xi using interplotion. 
    num_points : how big your covariance matrix is 

    Returns:
    Delta : Array of Noisy observations.
    
    """
    sigma = cov_matrix(sigma, num_points)
    noise = np.random.multivariate_normal(np.zeros(num_points), sigma)
    delta = observations_at_xi + noise 
    return delta




# Define Likelihood Function 

In [156]:
def phi(observations, predicted, sigma, num_points) :
    '''
    For a set of predetermined points xi -- obtained via np.linspace,
    this function defines the likelihood function 

    Arguments:
    observations: Generated noisy observation using beta_true -- corresponds to y in literature
    predicted: For a proposed beta_i, we compute the noisy observation using the forward solver 
    -- corresponds to g(beta_i) in literature

    Returns: 
    Likelihood function that is proportional to the prior distribution
    
    '''
    covariance_matrix = cov_matrix(sigma, num_points)
    diff = predicted - observations
    covariance_matrix_inv = (1/sigma**2) * covariance_matrix
    val = 0.5 * diff.T @ covariance_matrix_inv @ diff
    return val
# 2 cases, 1. denominator very close to 0 or both is 0. need to check if its one or both. 


In [157]:
def compute_A(observations, g_beta_current, g_beta_proposal, sigma, num_points):

    # val = np.exp((1/sigma**2)*(phi(observations, g_beta_current)- phi(observations, g_beta_proposal)))
    val = np.exp(phi(observations, g_beta_current, sigma, num_points) - phi(observations, g_beta_proposal, sigma, num_points))
    return val

# MCMC algorithm 

In [158]:
def MCMC(beta_true, number_of_iter, burn_in, sigma, num_points): 
    '''
    Builds a Markov Chain 

    Key Steps:
    1. Initialise a choice of Beta, beta_0 
    2. Compute likelihood of beta_0, using delta and beta_0_predicted
    3. Initialise the loop.
        - we propose a new beta_i from x* ~ Uniform(0.15, 0.85) and r ~ Uniform(0, 0.15)
        - compute y_i and g(beta_i)
        - compute likelihood using {y_i and g(beta_i)}
        - set alpha = min{1, likelihood }
    '''
    # set seed 
    np.random.seed(42)
    # range of uniform distribution 
    x_star_range = (0.15, 0.85)
    r_range = (0, 0.15)
    
    # compute delta 
    mesh_true , c_sol_true = sl.refinement_loop(0.00001, beta_true)
    y_true = interpolation_for_u_h(c_sol_true, mesh_true, num_points)
    delta = add_noise(y_true, num_points, sigma)

    # intiailise markov chain 
    chain = [] 
    beta_init = np.array([0.5, 1/4])
    beta_current = beta_init.copy()
    iter_count = 0
    acceptance_count = 0 
    acceptance_prob_history = []

    # initialise current observations and likelihood 
    mesh_current, c_sol_current = sl.refinement_loop(0.00001, beta = beta_current)
    y_current = interpolation_for_u_h(c_sol_current, mesh_current, num_points)


    for i in range(number_of_iter):
        beta_proposal = np.array([
            np.random.uniform(*x_star_range),
            np.random.uniform(*r_range)
        ])
        print("beta proposal:", beta_proposal)
        mesh_proposal, c_sol_proposal = sl.refinement_loop(0.00001, beta = beta_proposal)
        y_proposal = interpolation_for_u_h(c_sol_proposal, mesh_proposal, num_points)

        # compute acceptance probability 
        A = compute_A(delta, y_current, y_proposal, sigma, num_points)
        acceptance_prob = min(1, A)
        acceptance_prob_history.append(acceptance_prob)
        print("acceptance probablity:", acceptance_prob)
        
        # Accept or reject the proposal
        if np.random.rand() < acceptance_prob:
            beta_current = beta_proposal # update the current state as the last accepted proposal
            y_current = y_proposal # update the current observations to the last accepted observation
            acceptance_count += 1
        
        # Append the current beta to the chain
        chain.append(beta_current.copy())
        iter_count += 1
        print(iter_count)
    
    chain = np.array(chain)
    # # Discard burn-in and compute the MCMC estimate as the mean of the samples
    beta_mcmc = np.mean(chain[burn_in:], axis=0)
    return chain, beta_mcmc, acceptance_prob_history, acceptance_count

In [160]:
number_of_iter = 200
burn_in = 20
sigma = 0.1 
num_points = 100
beta_true = np.array([0.5, 1/6])

chain, beta_mcmc, acceptance_history, acceptance_count = MCMC(beta_true, number_of_iter, burn_in, sigma, num_points)
print("True beta:", beta_true )
print("MCMC estimated beta:", beta_mcmc)
print("Acceptance Probability History:", acceptance_history)
print("Acceptance Count:", acceptance_count)

beta proposal: [0.4421877  0.03331617]
acceptance probablity: 0.0231636787975203
1
beta proposal: [0.38633062 0.14143646]
acceptance probablity: 0.6162492303119662
2
beta proposal: [0.51315344 0.10545284]
acceptance probablity: 1
3
beta proposal: [0.83024746 0.14436709]
acceptance probablity: 2.7388101428656282e-05
4
beta proposal: [0.49807395 0.04513175]
acceptance probablity: 0.07043444442752295
5
beta proposal: [0.17582086 0.09143465]
acceptance probablity: 0.00025151447726193114
6
beta proposal: [0.18603513 0.04179697]
acceptance probablity: 0.0009319091281695639
7
beta proposal: [0.31769332 0.02173423]
acceptance probablity: 0.004504478198836502
8
beta proposal: [0.83995532 0.03630829]
acceptance probablity: 0.0006647862832490214
9
beta proposal: [0.68313373 0.03564563]
acceptance probablity: 0.009146486089735076
10
beta proposal: [0.40744819 0.09484587]
acceptance probablity: 0.393221633001661
11
beta proposal: [0.52504228 0.01354347]
acceptance probablity: 0.005394341595221232
1

In [ ]:
from concurrent.futures import ProcessPoolExecutor

def run_single_chain(seed, beta_true, number_of_iter, burn_in, sigma, num_points):
    # Set the random seed to ensure independent chains.
    np.random.seed(42)
    return MCMC(beta_true, number_of_iter, burn_in, sigma, num_points)

def run_multiple_chains(n_chains, beta_true, number_of_iter, burn_in, sigma, num_points):
    seeds = np.random.randint(0, 10000, size=n_chains)
    chains = []
    beta_mcmcs = []
    acceptance_histories = []
    acceptance_counts = []
    
    # Run chains in parallel using ProcessPoolExecutor
    with ProcessPoolExecutor() as executor:
        # Submit all the chains concurrently
        futures = [
            executor.submit(run_single_chain, seed, beta_true, number_of_iter, burn_in, sigma, num_points)
            for seed in seeds
        ]
        # Collect results as they complete
        for future in futures:
            chain, beta_mcmc, acceptance_prob_history, acceptance_count = future.result()
            chains.append(chain)
            beta_mcmcs.append(beta_mcmc)
            acceptance_histories.append(acceptance_prob_history)
            acceptance_counts.append(acceptance_count)
    
    return chains, beta_mcmcs, acceptance_histories, acceptance_counts

if __name__ == '__main__':
    n_chains = 4
    number_of_iter = 10000
    burn_in = 1000
    sigma = 0.1 
    num_points = 100
    beta_true = np.array([0.5, 1/6])
    chains, beta_mcmcs, acceptance_histories, acceptance_counts = run_multiple_chains(
        n_chains, beta_true, number_of_iter, burn_in, sigma, num_points
    )
print("True beta:", beta_true )
print("MCMC estimated beta:", beta_mcmcs)
print("Acceptance Probability History:", acceptance_histories)
print("Acceptance Count:", acceptance_counts)

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.